In [10]:
hosts="10.19.137.140,10.19.137.141,10.19.137.142,10.19.137.143,10.19.137.144,10.19.137.145,10.19.137.146,\
10.19.137.151,10.19.137.152,10.19.137.153,10.19.137.154,10.19.137.156,10.19.137.157,10.19.137.158,\
10.19.137.159,10.19.140.12,10.19.140.15,10.19.140.4,10.19.140.7,10.19.140.9,10.19.137.147,10.19.137.148,\
10.19.137.149,10.19.137.150".split(',')
master = ['10.19.137.140', '10.19.137.141', '10.19.137.142']
%matplotlib inline
import matplotlib.pyplot as plt

In [22]:
import re
import pandas as pd
import os
line_pattern = re.compile(r"(?P<type>[IWEF])\s*(?P<time>\d+ \d+:\d+:\d+\.\d+)\s*(?P<threadid>\d+)\s*(?P<filename>\S+):(?P<linenum>\d+)] (?P<message>.*)")
excludes=[
    "Log file created",
    "Running on machine",
    "Built with gc",
    "Log line format",
]
def collect_kubelet_logs():
    r = []
    for h in hosts:
        fname = os.path.join(".", "data", "kubelet", h, "kubelet.INFO")
        with open(fname) as f:
            for l in f:
                m = line_pattern.match(l)
                if not m:
                    ignore = False
                    for e in excludes:
                        if e in l:
                            ignore = True
                            break
                    if not ignore:
                        r.append({
                            "type": 'U',
                            "time": '0000 00:00:00.000000',
                            "threadid": "0",
                            "filename": "NA",
                            "linenum": "0",
                            "message": l.strip(),
                            "node": h,
                        })
                else:
                    v = m.groupdict()
                    v.update({"node": h})
                    r.append(v)
    return pd.DataFrame(r)

In [23]:
df = collect_kubelet_logs()

In [36]:
import numpy as np
pd.options.display.max_colwidth=100
def count(x):
    return x.shape[0]
def example(x):
    return x.iloc[0]

df[(df.type != 'I') & (df.type != 'U')].groupby(['filename', 'linenum'])['message'].agg([count, example]).sort_values(by="count", ascending=False).reset_index()

,filename,linenum,count,example
0,kubelet_volumes.go,128,1574352,"Orphaned pod ""0626453a-30ce-11e8-80de-1866da19caf3"" found, but volume paths are still present on..."
1,pod_workers.go,186,950281,"Error syncing pod a6c5b728-2c16-11e8-80de-1866da19caf3 (""csi-xfshostpath-plugin-vfkd4_kube-syste..."
2,reconciler.go,376,243479,Could not construct volume information: directory /data/kubelet/pods/0626453a-30ce-11e8-80de-186...
3,nestedpendingoperations.go,263,93268,"Operation for ""\""kubernetes.io/secret/a6c5b728-2c16-11e8-80de-1866da19caf3-sasecret\"" (\""a6c5b72..."
4,kubelet.go,1645,78660,"Unable to mount volumes for pod ""csi-xfshostpath-plugin-vfkd4_kube-system(a6c5b728-2c16-11e8-80d..."
5,csi_mounter.go,345,58244,kubernetes.io/csi: failed to open volume data file [/data/kubelet/pods/09295bd6-2c18-11e8-80de-1...
6,csi_plugin.go,164,58244,kubernetes.io/csi: plugin.ConstructVolumeSpec failed loading volume data using [/data/kubelet/po...
7,kubelet_pods.go,855,55374,Unable to retrieve pull secret 4tools/ for 4tools/enn-haproxy-ssr-7c6bbc94f4-r5c9z due to resour...
8,kuberuntime_manager.go,734,49707,container start failed: ErrImagePull: rpc error: code = Unknown desc = Error: image library/mypy...
9,mount_linux.go,170,40445,"could not determine device for path: ""/data/kubelet/pods/0626453a-30ce-11e8-80de-1866da19caf3/vo..."


In [49]:
# df[(df.type == 'U')].groupby("node").count().reset_index()
df[(df.type == 'U') & (df.node == '10.19.140.7')]['message']

5211347                                                                          Mounting command: systemd-run
5211348    Mounting arguments: --description=Kubernetes transient mount for /data/kubelet/pods/8e0f9c86-220...
5211349                             Output: Running scope as unit: run-r8ba406cb803e41cc93044fc700a051a0.scope
5211350    mount: special device 10.19.137.144:6789,10.19.137.145:6789,10.19.137.146:6789:/monitor-system-a...
5211351                                                                                                       
5211358                                                                          Mounting command: systemd-run
5211359    Mounting arguments: --description=Kubernetes transient mount for /data/kubelet/pods/8e0f9c86-220...
5211360                             Output: Running scope as unit: run-r6a3e884fb88d4f6fa946c0a9d51a0d31.scope
5211361    mount: special device 10.19.137.144:6789,10.19.137.145:6789,10.19.137.146:6789:/monitor-system-a...
5